멀테레이블 분류 - 클래스별 sigmoid 함수 사용

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import random
import os

import torch
from transformers import AutoTokenizer, AutoModel


def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42

# 현재 device 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
DATA_PATH = '/content/drive/MyDrive/2조_누구냐, 넌!/04. 기타자료/Naver_shopping_크롤링/Modeling/한수길/data/'

In [6]:
DATA_PATH_model = '/content/drive/MyDrive/2조_누구냐, 넌!/04. 기타자료/Naver_shopping_크롤링/Modeling/한수길/data/'

In [7]:
df = pd.read_csv(f"{DATA_PATH}real_labeling_topic.csv")
df

,content,score,date,brand,가격,세정력/거품,향기,트러블/진정,배송/서비스/고객대응,탈모 효과 및 쿨링감,머릿결*
0,배송누락되어 본사확신다고 하시고 다시 주문번호 올려달라고 하시니 대응절차가 문제있어...,1,23.08.31.,라보에이치,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,시원하고 좋은데 뾰루지 많이나고 따갑네요,1,23.09.07.,라보에이치,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,전에 쓰던 샴푸보다 머리카락 더빠짐…,1,23.03.25.,라보에이치,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,저랑은 안맞네요 계속 쓰다보니 두피 너무 가렵고 비듬각질이 생기네요,1,23.02.18.,라보에이치,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,효과는없고 계속많이빠져요,1,22.07.27.,라보에이치,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5915,머릿결이 좀 좋아졌어요 거품도 잘 나서 만족해요,5,21.12.12.,반코르,0.0,1.0,0.0,0.0,0.0,0.0,1.0
5916,1. 머리 덜 간지러움 2. 각질 비듬 줄어듬 3. 머리카락이 두꺼워지는게 느껴짐 ...,5,21.12.12.,반코르,0.0,0.0,0.0,1.0,0.0,1.0,0.0
5917,지성두피라서 오후만 되면 앞머리 떡지던 사람으로서 이정도로 만족스러운 샴푸는 처음입...,5,21.12.01.,반코르,0.0,1.0,0.0,0.0,0.0,0.0,1.0
5918,얼굴은 건성 두피은 지성인 말잇못 타입인데 추천의 추천을 받아서 구입했어요. 참고로...,5,21.11.10.,반코르,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5920 entries, 0 to 5919
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   content      5920 non-null   object 
 1   score        5920 non-null   int64  
 2   date         5920 non-null   object 
 3   brand        5920 non-null   object 
 4   가격           5920 non-null   float64
 5   세정력/거품       5920 non-null   float64
 6   향기           5920 non-null   float64
 7   트러블/진정       5920 non-null   float64
 8   배송/서비스/고객대응  5920 non-null   float64
 9   탈모 효과 및 쿨링감  5920 non-null   float64
 10  머릿결*         5920 non-null   float64
dtypes: float64(7), int64(1), object(3)
memory usage: 508.9+ KB


# target값 2차원 배열로 만들기

In [10]:
target = df.iloc[:,4:].to_numpy().reshape(-1,7)
target

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

# 사전학습 모델 kcelectra

In [11]:
model_name = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [12]:
train_token = tokenizer(df['content'].tolist(), padding="max_length", max_length=512, truncation=True)

In [13]:
input_ids = np.array(train_token['input_ids'])
attention_mask = np.array(train_token['attention_mask'])
token_type_ids = np.array(train_token['token_type_ids'])

input_ids.shape, attention_mask.shape, token_type_ids.shape

((5920, 512), (5920, 512), (5920, 512))

In [14]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, token_type_ids, y=None):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.y = y

    def __len__(self):
        return self.input_ids.shape[0]

    def __getitem__(self, idx):
        item = {}
        item['input_ids'] = torch.tensor(self.input_ids[idx])
        item['attention_mask'] = torch.tensor(self.attention_mask[idx])
        item['token_type_ids'] = torch.tensor(self.token_type_ids[idx])
        if self.y is not None:
            item['y'] = torch.Tensor(self.y[idx])

        return item

In [15]:
# dt = ReviewDataset(input_ids, attention_mask, token_type_ids, y=target)
# dl = torch.utils.data.DataLoader(dt,batch_size=2)
# batch = next(iter(dl))
# batch

In [16]:
class Net(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.output_layer = torch.nn.Linear( self.model.config.hidden_size , 7)

    def forward(self, input_ids, attention_mask, token_type_ids):
        x = self.model(input_ids, attention_mask, token_type_ids)
        x = self.output_layer(x[0][:,0])
        return x

In [17]:
# model = Net(model_name)
# x = model(batch["input_ids"], batch['attention_mask'], batch['token_type_ids'])
# x

In [18]:
# model.train()
# pred = model(batch["input_ids"].to(device), batch['attention_mask'].to(device), batch['token_type_ids'].to(device))

In [19]:
# pred

In [20]:
def train_loop(dataloader, model, bce_loss_fn, optimizer, device):
    epoch_loss = 0
    model.train() # 모델 객체를 학습모드로 전환
    for batch in tqdm(dataloader):
        pred = model(batch["input_ids"].to(device), batch['attention_mask'].to(device), batch['token_type_ids'].to(device))

        # multi-label Classification loss
        bce_loss = bce_loss_fn(pred , batch["y"].to(device))

        optimizer.zero_grad()
        bce_loss.backward()
        optimizer.step()

        epoch_loss += bce_loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [21]:
@torch.no_grad()
def test_loop(dataloader,model,bce_loss_fn, device):
    model.eval() # 평가 모드
    sig = torch.nn.Sigmoid()
    pred_list = []

    epoch_loss = 0
    for batch in tqdm(dataloader):
        pred = model(batch["input_ids"].to(device), batch['attention_mask'].to(device), batch['token_type_ids'].to(device))

        # 검증 평가할 경우
        if batch.get("y") is not None:
            bce_loss = bce_loss_fn(pred, batch["y"].to(device))

            epoch_loss += bce_loss.item()

        # 예측값 만들기
        pred = sig(pred).to("cpu").numpy()

        pred_list.append(pred)

    pred = np.concatenate(pred_list)
    epoch_loss /= len(dataloader)
    return epoch_loss, pred

In [22]:
!pip install iterative-stratification

In [23]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score

batch_size = 4
bce_loss_fn = torch.nn.BCEWithLogitsLoss()
epochs = 100
n_splits = 5
cv = MultilabelStratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)

In [25]:
is_holdout = False
reset_seeds(SEED)
best_score_list = []
best_recall_score_list = []
best_precision_score_list = []

for i, (tri, vai) in enumerate(cv.split(input_ids, target)):
    model = Net(model_name).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

    # 학습용
    train_dt = ReviewDataset(input_ids[tri], attention_mask[tri], token_type_ids[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)

    # 검증용
    valid_dt = ReviewDataset(input_ids[vai], attention_mask[vai], token_type_ids[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size, shuffle=False)

    best_score = 0
    best_recall_score = 0
    best_precision_score = 0


    patience = 0
    for epoch in tqdm(range(epochs)):
        train_loss = train_loop(train_dl, model, bce_loss_fn, optimizer, device)
        valid_loss, pred= test_loop(valid_dl, model, bce_loss_fn, device)

        # multi-label Classification
        pred = (pred > 0.5).astype(int)
        f1 = f1_score(target[vai], pred, average='macro')
        recall = recall_score(target[vai], pred, average='macro')
        precision = precision_score(target[vai], pred, average='macro')
        patience += 1

        if best_score < f1:
            patience = 0
            best_score = f1
            best_recall_score = recall
            best_precision_score = precision
            torch.save(model.state_dict(), f"{DATA_PATH_model}model_mlc_{i}.pth")

        print("f1_score_1:", f1)
        print("recall:", recall)
        print("precision:", precision)
        print("train_loss:", train_loss)
        print("valid_loss:", valid_loss)
        if patience == 3:
            break

    print(f"{i} 번째 폴드 best f1_score_1: {best_score}")
    print(f"{i} 번째 폴드 best recall_score: {best_recall_score}")
    print(f"{i} 번째 폴드 best precision_score: {best_precision_score}")
    best_score_list.append(best_score)
    best_recall_score_list.append(best_recall_score)
    best_precision_score_list.append(best_precision_score)
    if is_holdout:
        break

print(np.mean(best_score_list))
print(np.mean(best_recall_score_list))
print(np.mean(best_precision_score_list))

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1184 [00:00<?, ?it/s]

  0%|          | 0/296 [00:00<?, ?it/s]

f1_score_1: 0.8047987639340209
recall: 0.7867284215084294
precision: 0.8476060277582659
train_loss: 0.3375457009376102
valid_loss: 0.20923415740096085


  0%|          | 0/1184 [00:00<?, ?it/s]

...

  0%|          | 0/296 [00:00<?, ?it/s]

f1_score_1: 0.887499382024406
recall: 0.9081005532406892
precision: 0.8742442640104547
train_loss: 0.07927322517325347
valid_loss: 0.14546189121461497


  0%|          | 0/1184 [00:00<?, ?it/s]

  0%|          | 0/296 [00:00<?, ?it/s]

f1_score_1: 0.8817934116662304
recall: 0.9034579189796957
precision: 0.8648752472553953
train_loss: 0.061146910799465874
valid_loss: 0.1545823606964503
4 번째 폴드 best f1_score_1: 0.8892428037120462
4 번째 폴드 best recall_score: 0.9154518727404708
4 번째 폴드 best precision_score: 0.8674686292941285
0.8920064280112765
0.9169784818579398
0.8704226999968665
